In [1]:
import transformers
from datasets import load_dataset
import torch

import sys
from transformers import AutoTokenizer, AutoModelForCausalLM

model_path = "/content/saved-model/"
trained_model_path = "/content/model"

tokenizer = AutoTokenizer.from_pretrained(model_path, local_files_only=True)
model = AutoModelForCausalLM.from_pretrained(
            model_path, torch_dtype=torch.float16, load_in_8bit=True, device_map="auto", trust_remote_code=True)



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda117.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.9
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('443'), PosixPath('//10.104.0.1'), PosixPath('tcp')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('tiiuae/falcon-7b-instruct')}
  warn(msg)
/usr/local/lib/python3.10/dist-

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Prompt before fine tuning

In [2]:
def promptTemplate(instruction):
    return "{0}\n\n{1}\n{2}\n\n{3}\n".format(
        "Below is an instruction that describes a task. Write a response that appropriately completes the request.",
        "### Instruction:",
        instruction,
        "### Response:"
    )

text = promptTemplate("Write YAML that defines a Kubernetes Deployment named \"iis\" with 3 replicas ")
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)
if "token_type_ids" in inputs:
    inputs.pop("token_type_ids")
outputs = model.generate(**inputs, max_new_tokens=400)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1264: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Write YAML that defines a Kubernetes Deployment named "iis" with 3 replicas 

### Response:
apiVersion: v1
kind: Deployment
metadata:
  name: iis
  labels:
    app: iis
spec:
  replicas: 3
  selector:
    app: iis
  template:
    metadata:
      labels:
        app: iis
    spec:
      containers:
      - name: iis-container
        image: iis-image
        ports:
        - containerPort: 80

The YAML code provided defines a Kubernetes Deployment named "iis" with 3 replicas. The "metadata" section specifies the name of the deployment and the labels that apply to it. The "spec" section defines the desired number of replicas and the container image to use.


In [3]:
import os 

data = load_dataset("json", data_files="/content/data/*.jsonl")
data

Found cached dataset json (/root/.cache/huggingface/datasets/json/default-2d69f16079490881/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 282
    })
})

In [4]:

def prompt(instruction, output):
    prompt = "{0}\n\n{1}\n{2}\n\n{3}\n{4}".format(
        "Below is an instruction that describes a task. Write a response that appropriately completes the request.",
        "### Instruction:",
        instruction,
        "### Response:",
        output
    )
    return prompt

In [5]:
from peft import get_peft_model, LoraConfig, prepare_model_for_kbit_training

lora_config2 = LoraConfig(
 r=16,
 lora_alpha=32,
 target_modules=["query_key_value"],
 lora_dropout=0.05,
 bias="none",
 task_type="CAUSAL_LM"
)

model = prepare_model_for_kbit_training(model)

# add LoRA adaptor
model = get_peft_model(model, lora_config2)
model.print_trainable_parameters()

trainable params: 4,718,592 || all params: 6,926,439,296 || trainable%: 0.06812435363037071


In [6]:
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
print(data)
data = data.map(lambda x: tokenizer(prompt(x["prompt"], x["completion"]),
    max_length=1000, padding=True, truncation=True))
print("After tokenizing:", data)

Loading cached processed dataset at /root/.cache/huggingface/datasets/json/default-2d69f16079490881/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/cache-7224d2ad124fbca0.arrow


DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 282
    })
})
After tokenizing: DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 282
    })
})


In [7]:
epochs = int(os.environ.get("PARAM_EPOCHS", 1))

trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        num_train_epochs=epochs,
        # max_steps=5,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="./outputs",
        optim="paged_adamw_32bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

trainer.save_model(trained_model_path)

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:321: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
1,1.749700
2,1.570100
3,1.678600
4,1.652600
5,1.571400
6,1.336300
7,1.410100
8,1.619600
9,1.407600
10,1.249800


In [8]:
! nvidia-smi
! ls -lash {trained_model_path}

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Wed Jul  5 22:32:35 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA L4           Off  | 00000000:00:04.0 Off |                    0 |
| N/

In [9]:
text = promptTemplate("Write YAML that defines a Kubernetes Deployment named \"iis\" with 3 replicas ")
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)
if "token_type_ids" in inputs:
    inputs.pop("token_type_ids")

outputs = model.generate(**inputs, max_new_tokens=400)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Write YAML that defines a Kubernetes Deployment named "iis" with 3 replicas 

### Response:
```yaml
apiVersion: v1
kind: Deployment
metadata:
  name: iis
spec:
  replicas: 3
  selector:
    matchLabels:
      app: iis
  template:
    metadata:
      labels:
        app: iis
    spec:
      containers:
      - name: iis
        image: microsoft.azure.iis
        ports:
        - containerPort: 80

```


In [10]:
print(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): RWForCausalLM(
      (transformer): RWModel(
        (word_embeddings): Embedding(65024, 4544)
        (h): ModuleList(
          (0-31): 32 x DecoderLayer(
            (input_layernorm): LayerNorm((4544,), eps=1e-05, elementwise_affine=True)
            (self_attention): Attention(
              (maybe_rotary): RotaryEmbedding()
              (query_key_value): Linear8bitLt(
                in_features=4544, out_features=4672, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4544, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4672, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): Paramet

In [11]:
# model = model.merge_and_unload()
# this would throw the following error Cannot merge LORA layers when the model is loaded in 8-bit mode
# that's why we reload base model in 16 bit below

ValueError: Cannot merge LORA layers when the model is loaded in 8-bit mode

In [12]:
peft_model = model
model = AutoModelForCausalLM.from_pretrained(
            model_path, torch_dtype=torch.float16, load_in_8bit=False, device_map="auto", trust_remote_code=True)
print(model)
print(peft_model)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

RWForCausalLM(
  (transformer): RWModel(
    (word_embeddings): Embedding(65024, 4544)
    (h): ModuleList(
      (0-31): 32 x DecoderLayer(
        (input_layernorm): LayerNorm((4544,), eps=1e-05, elementwise_affine=True)
        (self_attention): Attention(
          (maybe_rotary): RotaryEmbedding()
          (query_key_value): Linear(in_features=4544, out_features=4672, bias=False)
          (dense): Linear(in_features=4544, out_features=4544, bias=False)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): MLP(
          (dense_h_to_4h): Linear(in_features=4544, out_features=18176, bias=False)
          (act): GELU(approximate='none')
          (dense_4h_to_h): Linear(in_features=18176, out_features=4544, bias=False)
        )
      )
    )
    (ln_f): LayerNorm((4544,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=4544, out_features=65024, bias=False)
)
PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): RWForC

In [13]:
! nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Wed Jul  5 22:34:55 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA L4           Off  | 00000000:00:04.0 Off |                    0 |
| N/

In [14]:
from peft import PeftModel

peft_model = PeftModel.from_pretrained(model, trained_model_path)

print(peft_model)



PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): RWForCausalLM(
      (transformer): RWModel(
        (word_embeddings): Embedding(65024, 4544)
        (h): ModuleList(
          (0-31): 32 x DecoderLayer(
            (input_layernorm): LayerNorm((4544,), eps=1e-05, elementwise_affine=True)
            (self_attention): Attention(
              (maybe_rotary): RotaryEmbedding()
              (query_key_value): Linear(
                in_features=4544, out_features=4672, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4544, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4672, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict

In [15]:
model = peft_model.merge_and_unload()

print(model)


RWForCausalLM(
  (transformer): RWModel(
    (word_embeddings): Embedding(65024, 4544)
    (h): ModuleList(
      (0-31): 32 x DecoderLayer(
        (input_layernorm): LayerNorm((4544,), eps=1e-05, elementwise_affine=True)
        (self_attention): Attention(
          (maybe_rotary): RotaryEmbedding()
          (query_key_value): Linear(in_features=4544, out_features=4672, bias=False)
          (dense): Linear(in_features=4544, out_features=4544, bias=False)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): MLP(
          (dense_h_to_4h): Linear(in_features=4544, out_features=18176, bias=False)
          (act): GELU(approximate='none')
          (dense_4h_to_h): Linear(in_features=18176, out_features=4544, bias=False)
        )
      )
    )
    (ln_f): LayerNorm((4544,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=4544, out_features=65024, bias=False)
)


In [16]:
! rm {trained_model_path}/*
model.save_pretrained(trained_model_path)
tokenizer.save_pretrained(trained_model_path)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [17]:
! ls -lash {trained_model_path}

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
total 13G
4.0K drwxr-xr-x 1 root root 4.0K Jul  5 22:35 .
8.0K drwxr-xr-x 1 root root 4.0K Jul  5 10:00 ..
4.0K -rw-r--r-- 1 root root  707 Jul  5 22:35 config.json
4.0K -rw-r--r-- 1 root root  116 Jul  5 22:35 generation_config.json
9.3G -rw-r--r-- 1 root root 9.3G Jul  5 22:35 pytorch_model-00001-of-00002.bin
3.7G -rw-r--r-- 1 root root 3.7G Jul  5 22:35 pytorch_model-00002-of-00002.bin
 20K -rw-r--r-- 1 root root  17K Jul  5 22:35 pytorch_model.bin.index.json
